In [252]:
import glob
import os
import pandas as pd
import folium
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from parse_arduino_data import parse_adafruit_latlon
from geopy.distance import geodesic
import numpy as np

sns.set(style='white')

# Load main path data

In [2]:
lines = []
with open('data/principal-shape.nmea', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        rowtype = row[0]
        if rowtype == '$GPGGA':
            rowtype, utc, lat, lat_dir, lon, lon_dir, quality, nsats, hdop, alt, a_units, undulation, u_units, *diff_data, check = row
            assert rowtype == '$GPGGA'
            assert a_units == u_units == 'M'
            line = [utc, str(lat) + lat_dir, str(lon) + lon_dir, quality, nsats, hdop, alt]
        elif rowtype == '$GPRMC':
            rowtype, utc, status, lat, lat_dir, lon, lon_dir, speed, track_true, date, *stuff, check = row
            assert rowtype == '$GPRMC'
            line.extend([status, speed, date])
            lines.append(line)
            line = None

trail = pd.DataFrame(lines, columns=['utc', 'lat', 'lon', 'quality', 'nsats', 'hdop', 'alt', 'status', 'speed', 'date'])
trail.query('status == "A"', inplace=True)
trail['lat'] = trail['lat'].map(parse_adafruit_latlon)
trail['lon'] = trail['lon'].map(parse_adafruit_latlon)
trail['hdop'] = trail['hdop'].astype(float)
trail.query('hdop < 2', inplace=True)

# Manual loop for trimming data

In [3]:
files = glob.glob('data/*below*.csv')
idx = 0
with open('data/below-trim.csv', 'w') as f:
    f.write('fname,start,stop\n')

below_fname = files[idx]
below = pd.read_csv(below_fname)
below.sort_values('boardtime', inplace=True)
try:
    laps = pd.read_excel(below_fname.replace('below', 'laps').replace('csv', 'xlsx'), header=None)[0]
    haslaps = True
except FileNotFoundError:
    haslaps = False
    pass
else:
    print(laps.shape)
    print(f'({laps.values[0]}, {laps.values[-1]})')
below['distance'] = below[['lat', 'lon']].apply(lambda dp: trail[['lat', 'lon']].apply(lambda tp: geodesic(dp, tp).meters, axis=1).min(), axis=1)

In [ ]:
start, stop = 509000, 899000
ax = below.query('boardtime >= @start and boardtime <= @stop').plot.scatter(x='boardtime', y='distance')
below.query('boardtime < @start or boardtime > @stop').plot.scatter(x='boardtime', y='distance', color='black', ax=ax)
ax.axvline(start, color='red')
ax.axvline(stop, color='green')
ax.figure.set_size_inches(10, 5)

print(below_fname)

def timetomillis(t):
    m, s = t.split(':')
    return (int(m) * 60 + int(s)) * 1000

if haslaps:
    print(laps.shape)
    print(f'({timetomillis(laps.values[0])}, {timetomillis(laps.values[-1])})')

print(below.query('boardtime >= @start')['distance'].values[0],
      below.query('boardtime <= @stop')['distance'].values[-1])

if False:
    m = folium.Map(
        tiles="Stadia.AlidadeSatellite",
        location=[18.448501, -66.592225],
        zoom_start=16,
        zoom_control=False,
    )
    for lat, lon in trail[['lat', 'lon']].values:
        folium.CircleMarker(
            location=[lat, lon],
            radius=3,
            fill_opacity=1,
            fill=True,
            stroke=False,
            color='purple',
        ).add_to(m)
    for lat, lon in below[['lat', 'lon']].values:
        folium.CircleMarker(
            location=[lat, lon],
            radius=3,
            fill_opacity=1,
            fill=True,
            stroke=False,
            color='green',
        ).add_to(m)
else:
    m = None
m

In [219]:
with open('data/below-trim.csv', 'a') as f:
    f.write(f'{below_fname},{start},{stop}\n')

idx += 1

if idx < len(files):
    below_fname = files[idx]
    below = pd.read_csv(below_fname)
    below.sort_values('boardtime', inplace=True)
    try:
        laps = pd.read_excel(below_fname.replace('below', 'laps').replace('csv', 'xlsx'), header=None)[0]
        haslaps = True
    except FileNotFoundError:
        haslaps = False
        pass
    below['distance'] = below[['lat', 'lon']].apply(lambda dp: trail[['lat', 'lon']].apply(lambda tp: geodesic(dp, tp).meters, axis=1).min(), axis=1)
else:
    print('done!')

done!


# Put distances in all files

In [229]:
for fname in glob.glob('data/*.csv'):
    df = pd.read_csv(fname)
    if 'lat' in df.columns:
        df['distance'] = df[['lat', 'lon']].apply(lambda dp: trail[['lat', 'lon']].apply(lambda tp: geodesic(dp, tp).meters, axis=1).min(), axis=1)
        df.to_csv(fname, index=False)

# Plot trimmed below-data

In [ ]:
trim = pd.read_csv('data/below-trim.csv')
for fname, start, stop in trim.values:
    below = pd.read_csv(fname)
    ax = below.query('boardtime >= @start and boardtime <= @stop').plot.scatter(x='boardtime', y='distance')
    below.query('boardtime < @start or boardtime > @stop').plot.scatter(x='boardtime', y='distance', color='black', ax=ax)
    ax.axvline(start, color='red')
    ax.axvline(stop, color='green')
    ax.figure.set_size_inches(10, 5)
    ax.set_title(fname)
    plt.tight_layout()
    ax.figure.savefig(f'below-trim-plots/{fname.split("/")[1].replace("csv", "png")}')

# Find laps suitable for determining soil distances

In [248]:
from collections import defaultdict
flag_coords = defaultdict(lambda: (0, 0, float('inf')))  # maps (transect, flagnum) to (lat, lon, hdop)

m = folium.Map(
    tiles="Stadia.AlidadeSatellite",
    location=[18.448501, -66.592225],
    zoom_start=16,
    zoom_control=False,
)
for lat, lon in trail[['lat', 'lon']].values:
    folium.CircleMarker(
        location=[lat, lon],
        radius=3,
        fill_opacity=1,
        fill=True,
        stroke=False,
        color='purple',
    ).add_to(m)

for fname, start, stop in trim.values:
    if '_1_' in fname:
        transect = 1
    elif '_2_' in fname:
        transect = 2
    elif '_3_' in fname:
        transect = 3
    else:
        raise Exception(fname)
    
    laps_fname = fname.replace('below', 'laps').replace('csv', 'xlsx')
    if os.path.exists(laps_fname):
        below = pd.read_csv(fname)
        laps = pd.read_excel(laps_fname, header=None)[0]
        if len(laps) < 31:
            continue
        first, last = laps.values[[0, -1]]
        if timetomillis(first) == start and timetomillis(last) == stop:
            for flagnum in range(16):
                lap = timetomillis(laps.iloc[flagnum])
                first_point_after = below.query('boardtime > @lap + 1').iloc[0]
                folium.CircleMarker(
                    location=first_point_after[['lat', 'lon']].values,
                    radius=3,
                    fill_opacity=1,
                    fill=True,
                    stroke=False,
                    color=f'#{flagnum * 16:02x}0000',
                ).add_to(m)
                if first_point_after['hdop'] < flag_coords[transect, flagnum][2]:
                    flag_coords[transect, flagnum] = first_point_after[['lat', 'lon', 'hdop']].values

m

In [265]:
items = []
for (transect, flag), (lat, lon, hdop) in flag_coords.items():
    distance = trail[['lat', 'lon']].apply(lambda tp: geodesic((lat, lon), tp).meters, axis=1).min()
    items.append([transect, flag, distance])
flag_distances = pd.DataFrame(items, columns=['transect', 'flag', 'distance']).set_index(['transect', 'flag'])
flag_distances

distance
transect flag            
1        0       0.749212
         1       7.478920
         2      15.246417
         3      25.205102
         4      31.368173
         5      42.045261
         6      50.004735
         7      57.849051
         8      61.797594
         9      69.269396
         10     76.162274
         11     86.340281
         12     92.364175
         13     94.257835
         14     93.287155
         15     90.685158
2        0       3.648216
         1       9.083264
         2      16.508450
         3      28.150991
         4      39.362256
         5      51.229794
         6      57.451772
         7      65.872790
         8      77.234476
         9      89.108228
         10     98.453113
         11    107.074881
         12    119.162132
         13    128.759325
         14    136.731894
         15    145.113142
3        0      10.202499
         1      10.020301
         2      17.563859
         3      21.611830
         4      27.750103
         5      29.300819
         6      38.475054
         7      48.369286
         8      58.496836
         9      67.329529
         10     79.850331
         11     91.195493
         12     98.323530
         13    107.588840
         14    118.372638
         15    129.220697

In [266]:
flag_distances.to_csv('data/flag-distances.csv')